# Merging the NIRs data with text columns

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings; warnings.filterwarnings('ignore')

# all data
data_path = '../data'

# nirs data
nirs_data_sub1 = os.path.join(data_path, 'neuro/nirs_sub1.csv')
nirs_data_sub2 = os.path.join(data_path, 'neuro/nirs_sub2.csv')

# text data
ceda_data_path = os.path.join(data_path, 'results/ceda-results.csv')
########## may need to change . . . I need dyads->speakers data.
meta_data_path = os.path.join(data_path, 'meta_data/CC_Post_105dyads.csv')

### Import main data

In [ ]:
df = pd.read_csv(ceda_data_path)
df.head()

### Dealing with TimeStamps

In [ ]:
def to_NIRs_freq(t):
    timestamp_pieces = t.split(':')
    t = float(timestamp_pieces[-1])
    if len(timestamp_pieces) > 1:
        t+= (float(timestamp_pieces[-2]) * 60)
    if len(timestamp_pieces) > 2:
        t += (float(timestamp_pieces[-3]) * (60 * 60))
    
    return t * 5.0863

In [ ]:
for dyad in tqdm(df['file'].unique()):
    sub = df.loc[df['file'].isin([dyad])]
    
    if sub['timestamp'].loc[sub.index[0]] in [np.nan, None]:
        sub['timestamp'].loc[sub.index[0]] = '00:00'
        df['timestamp'].loc[sub.index[0]] = '00:00'
    
    no_markers = sub.loc[sub['timestamp'].isna()].index
    has_marker = sub.loc[~sub['timestamp'].isna()].index
    
    for i in no_markers:
        marker = has_marker[has_marker<i]
        if len(marker) > 0:
            marker = marker[-1]
            df['timestamp'].loc[i] = df['timestamp'].loc[marker]
        
        # elif (i == sub.index[0]):
        #     df['timestamp'].loc[i] = '00:00'
        
df.head(20)

In [ ]:
df['NIRS_time'] = [to_NIRs_freq(t) for t in tqdm(df['timestamp'].values)]

In [ ]:
df[['file', 'NIRS_time']].head()

In [ ]:
df['next_NIRS_time'] = None

for dyad in tqdm(df['file'].unique()):
    sub = df.loc[df['file'].isin([dyad])]
    for i in sub.index:
        t_ = sub['NIRS_time'].loc[i]
        s_ = sub['NIRS_time'].loc[sub['NIRS_time'] > t_].values
        
        if len(s_) > 0:
            df['next_NIRS_time'].loc[i] = s_[0]
        
        elif i == sub.index[-1]:
            df['next_NIRS_time'].loc[i] = 1e9
        
        else:
            df['next_NIRS_time'].loc[i] = sub['NIRS_time'].max()
        
df[['file','NIRS_time', 'next_NIRS_time']].head(20)

In [ ]:
df['next_next_NIRS_time'] = None

for dyad in tqdm(df['file'].unique()):
    sub = df.loc[df['file'].isin([dyad])]
    for i in sub.index:
        t_ = sub['NIRS_time'].loc[i]
        s_ = sub['NIRS_time'].loc[sub['NIRS_time'] > t_].unique()
        
        if len(s_) > 1:
            df['next_next_NIRS_time'].loc[i] = s_[1]
        
        elif i == sub.index[-1]:
            df['next_next_NIRS_time'].loc[i] = 1e9
        
        else:
            df['next_next_NIRS_time'].loc[i] = sub['next_NIRS_time'].loc[i]
        
df[['file','NIRS_time', 'next_NIRS_time', 'next_next_NIRS_time']].head(20)

In [ ]:
df['NIRS_time'] = [np.ceil(t) for t in tqdm(df['NIRS_time'].values)]
df['next_NIRS_time'] = [np.floor(t) for t in tqdm(df['next_NIRS_time'].values)]
df['next_next_NIRS_time'] = [np.floor(t) for t in tqdm(df['next_next_NIRS_time'].values)]
df[['file','NIRS_time', 'next_NIRS_time', 'next_next_NIRS_time']].head(20)

### Process

Per conversation ID, streaming fNIRs data and grabbing the appropriate speakers.

In [ ]:
unified_neuro_home = os.path.join(data_path, 'unified_neuro')
fnirs_files = [
    os.path.join(unified_neuro_home, f) for f in os.listdir(unified_neuro_home) 
    if (not f.startswith('._'))
]
# fnirs_files

In [ ]:
channel_indexes = {
    "L_lPFC": ['ch_'+str(i) for i in range(1,6+1)],
    "mPFC": ['ch_'+str(i) for i in range(7,14+1)],
    "R_lPFC": ['ch_'+str(i) for i in range(15,20+1)],
    "L_SPL": ['ch_'+str(i) for i in range(21,23+1)],
    "L_TPJ": ['ch_'+str(i) for i in range(24,30+1)],
    "R_SPL": ['ch_'+str(i) for i in range(31,33+1)],
    "R_TPJ": ['ch_'+str(i) for i in range(34,40+1)],
}

In [ ]:
channel_to_region_names = dict()
for k, v in channel_indexes.items():
    for i,channel in enumerate(v):
        channel_to_region_names[channel] = k + '_ch_' + str(i+1)

In [ ]:
all_channels = sum(channel_indexes.values(), [])
df[all_channels] = None

for dyad in tqdm(df['file'].unique()):
    sub = df.loc[df['file'].isin([dyad])].index.values
    fnirs_data = [f for f in fnirs_files if str(dyad) in f][0]
    fnirs_data = pd.read_csv(fnirs_data)

    # make the next time stamp for all the conversation, for the last turn, 
    #  be the end of the conversation...
    # if df['next_NIRS_time'].loc[sub[-1]] == df['next_next_NIRS_time'].loc[sub[-1]]:
    #     df['next_NIRS_time'].loc[sub[-1]] = len(fnirs_data)
    df['next_next_NIRS_time'].loc[sub[-1]] = len(fnirs_data)
    
    if df['next_NIRS_time'].loc[sub[-1]] in [np.nan, None]:
        df['next_NIRS_time'] = len(fnirs_data)

    ## DO SOMETHING
    for start_t, end_t in df[['NIRS_time','next_NIRS_time']].loc[sub].drop_duplicates().values:
        values = fnirs_data[all_channels].loc[
            (fnirs_data['Time'] >= start_t)
            & (fnirs_data['Time'] < end_t)
        ]
        
        mu = np.nan_to_num(values.mean(axis=0).values)
        
        sel = df['NIRS_time'].isin([start_t]) & df['next_NIRS_time'].isin([end_t]) & df.index.isin(sub)
        
        for i in range(mu.shape[0]):
            df[all_channels[i]].loc[sel] = mu[i]


In [ ]:
df[['file', 'nx', 'ny', 'Hxy']+all_channels[:5]].head()

In [ ]:
all_new_channels = ['next_'+col for col in sum(channel_indexes.values(), [])]
df[all_new_channels] = None

for dyad in tqdm(df['file'].unique()):
    sub = df.loc[df['file'].isin([dyad])].index.values
    fnirs_data = [f for f in fnirs_files if str(dyad) in f][0]
    fnirs_data = pd.read_csv(fnirs_data)

    ## DO SOMETHING
    for start_t, end_t in df[['NIRS_time','next_next_NIRS_time']].loc[sub].drop_duplicates().values:
        values = fnirs_data[all_channels].loc[
            (fnirs_data['Time'] >= start_t)
            & (fnirs_data['Time'] < end_t)
        ]
        
        mu = values.mean(axis=0).values
        
        sel = df['NIRS_time'].isin([start_t]) & df['next_next_NIRS_time'].isin([end_t]) & df.index.isin(sub)
        
        for i in range(mu.shape[0]):
            df[all_new_channels[i]].loc[sel] = mu[i]


In [ ]:
df[['file', 'nx', 'ny', 'Hxy']+all_new_channels[:5]].head()

In [ ]:
df = df.rename(columns=channel_to_region_names)

### next_ region names
channel_to_region_names2 = dict()
for k,v in channel_to_region_names.items():
    channel_to_region_names2['next_'+k] = 'next_'+v
df = df.rename(columns=channel_to_region_names2)

# [col for col in list(df) if '_ch_' in col]

In [ ]:
df.to_csv(ceda_data_path.replace('.csv', '-with_fNIRs-long.csv'), index=False, encoding='utf-8')